In [3]:
"""
Created on Tue Sep 17 13:17:44 2019

@author: MatthewOliver
"""

"""
API calls to OMDb for tv_show data.
Matt Oliver Taylor Appel
"""
import csv
import numpy as np
import omdb
import time
import pandas as pd
import mysql.connector
from config import pword, host, user, api_key
import Project_1_Webscrape_Functions as web

# define omdb client for API calls
client=omdb.OMDBClient(apikey=api_key)

def omdb_call(tv_title, tv_year):
    '''
    takes in a key to the api, a title of a tv show, and the year it was released
    returns all the imdb data from the omdb API
    '''
    int_year=int(tv_year)
    data = client.get(title=tv_title,  #omdb.OMDBC requires a tv title and a release year
                      year=tv_year,
                      tomatoes=False) #tomatoes is a parameter that returns rotten tomatoes data
    return data

def get_tvshow_data(listy,year):
    '''
    takes in a list of tv shows released in the same year, and a key to the api,
    returns a Pandas Data Frame of all the imdb info on the shows
    '''
    results=[]
    for i, show in enumerate(listy):
        print(str(i)+show)
        results.append(omdb_call(show,year))
        time.sleep(.1) #wait 50 milliseconds
    return pd.DataFrame(results)

def ship_2_sql(dflisty):
    '''
    takes in a tv show dataframe then sends the data into AWS tv show database
    first doing the merchendise data
    '''
    cnx = mysql.connector.connect(
    #connect to mysql platform        
        host =host,
        user =user,
        passwd = pword
        )
    cur=cnx.cursor()
    cur.execute('USE tv_merch')
    for i, show in dflisty.iterrows():
        print(i)
    # first pass the show data into sql
#         cur.execute("""INSERT INTO show_data (
#                                     show_id,
#                                     show_name,
#                                     genre,
#                                     imdb_rating,
#                                     imdb_votes,
#                                     metascore,
#                                     released,
#                                     rated
#                                     ) 
#         VALUES ("{}","{}","{}","{}","{}","{}","{}","{}"
#         )""".format(show['imdb_id'],
#                     show['title'],
#                     show['genre'],
#                     show['imdb_rating'],
#                     show['imdb_votes'],
#                     show['metascore'],
#                     show['year'],
#                     show['rated']))
#     then pass the sales data into sql
        cur.execute("""INSERT INTO merch_data (
                                    show_id,
                                    etsy,
                                    ebay,
                                    bonanza,
                                    show_name
                                    ) 
        VALUES ({},{},{},{},{}
        )""".format(show['imdb_id'],
                    show['etsy'],
                    show['ebay'],
                    show['bonanza'],
                    show['title']))
    cnx.commit()
    cur.close()
    cnx.close()
    

    
    

'''
#test code
show_name = str(input('Name: \n'))
show_year = str(input('Year: \n'))

df = pd.DataFrame(omdb_call( api_key, show_name,show_year))
df.head(1)
'''





#brings all programs together, creating cumulative dataframe and saving to SQL

'''
shows_by_year=web.tv_list_makr()
imdb_data_by_year=[]
# for i, year in enumerate(shows_by_year[9:]):
#     imdb_data_by_year.append(get_tvshow_data(year,2019+i))
frames=[                imdb_data_by_year[1],
                        imdb_data_by_year[2],
                        imdb_data_by_year[3],
                        imdb_data_by_year[4],
                        imdb_data_by_year[5],
                        imdb_data_by_year[6],
                        imdb_data_by_year[7],
                        imdb_data_by_year[8],
                        imdb_data_by_year[9],
                        imdb_data_by_year[10]]
show_data=pd.concat(frames)
show_data['year']
'''


#quick look for element in merch data 
'''
data.drop([86],inplace=True) #unwanted column
j=3
print(data['title'][j])
print(bonanza[j]['result'][0]['Results'])
print(ebay[j]['result'][0]['Results'])
print(etsy[j]['result'][0]['Results'])
'''


#import ebay, etsy and bonanza data into show dataframe
'''
result=lambda x:x['result'][0]['Results']
data['ebay']=[result(ebay[i]) for i in range(len(ebay))]
data['etsy']=[result(etsy[i]) for i in range(len(etsy))]
data['bonanza']=[result(bonanza[i]) for i in range(len(bonanza))]
'''

#dropping unwanted columns
'''
drop_me=['awards',
         'actors',
         'box_office',
         'country',
         'director',
         'dvd',
         'language',
         'metascore',
         'plot',
         'poster',
         'production',
         'response',
         'total_seasons',
         'ratings',
         'type',
         'website',
         'writer']
data.drop(drop_me, axis=1,inplace=True)
'''

#save dataframe
'''
data=pd.read_csv('tv_show_data.csv')
'''

#data exploration tool
gimme= ['title','ebay','etsy','bonanza','imdb_votes','imdb_rating','year']
data[gimme].sort_values(['ebay'],ascending=False)[200:250]
    
#cleaning etsy data
etsy_filter=lambda x,y,z: np.NaN if (x>100)and(z<=4)and(y<10) else x
etsy_high_clip=lambda x: 0 if x>40000 else x
data['etsy2']=[etsy_medium_clip(data['etsy'][i],data['bonanza'][i],data['ebay'][i]) for i in range(len(data['etsy']))]

#change column type
data['imdb_votes']=data['imdb_votes'].map(lambda x: x.replace(',','')if type(x)==str else x)
data['imdb_votes']=data['imdb_votes'].astype(dtype='float64',errors='raise')



"\n#test code\nshow_name = str(input('Name: \n'))\nshow_year = str(input('Year: \n'))\n\ndf = pd.DataFrame(omdb_call( api_key, show_name,show_year))\ndf.head(1)\n"

In [216]:
data.dtypes

genre           object
imdb_id         object
imdb_rating    float64
imdb_votes      object
rated           object
released        object
runtime         object
title           object
year            object
ebay             int64
etsy             int64
bonanza          int64
etsy2          float64
dtype: object

In [243]:
gimme= ['title','ebay','etsy2','bonanza','imdb_votes','imdb_rating','year']
data[gimme].sort_values(['imdb_votes'],ascending=False)[0:50]

,title,ebay,etsy2,bonanza,imdb_votes,imdb_rating,year
21,The Walking Dead,3261,602.0,105,796251.0,8.3,2010–
298,Stranger Things,2497,1432.0,55,652903.0,8.8,2016–
178,True Detective,86,52.0,1,453111.0,9.0,2014–
58,Arrow,3212,155.0,63,383655.0,7.6,2012–
305,Westworld,1196,60.0,5,359026.0,8.8,2016–
126,Vikings,2940,233.0,4,352516.0,8.6,2013–
51,Suits,848,343.0,6,333327.0,8.5,2011–
116,Rick and Morty,1234,170.0,12,287285.0,9.3,2013–
37,Homeland,119,23.0,8,282965.0,8.3,2011–
216,Mr. Robot,871,77.0,8,282792.0,8.5,2015–


In [239]:
data['imdb_votes']=data['imdb_votes'].map(lambda x: x.replace(',','')if type(x)==str else x)
data['imdb_votes']=data['imdb_votes'].astype(dtype='float64',errors='raise')

In [286]:
data['genre'][1][1]

' Reality-TV'

In [287]:
data['genre']

0                                                                             [Drama]
1                                                          [Documentary,  Reality-TV]
2                                                                       [Documentary]
3                                                [Crime,  Drama,  Mystery,  Thriller]
4                                                        [Comedy,  Music,  Talk-Show]
5                                                               [Documentary,  Crime]
6                                                                        [Reality-TV]
7                                                            [Game-Show,  Reality-TV]
8                                                            [Action,  Crime,  Drama]
9                                                                       [Documentary]
10                                                                 [News,  Talk-Show]
11                                                    

In [295]:
genres=[]
for i, show in data.iterrows():
    if type(data['genre'][i])==list:
        for j in range(len(data['genre'][i])):
            genres.append(data['genre'][i][j])
#             show['imdb_id']
#             print(i,j,data['genre'][i][j])
    else:
        pass

In [290]:
spltme= lambda x:x.strip if type(x)==str else x

In [284]:
data['genre']=data['genre'].map(make_lst)

In [300]:
x = np.array(genres) 
print(np.unique(x))
print(len(np.unique(x)))

['Action' 'Adventure' 'Animation' 'Biography' 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Family' 'Fantasy' 'Game-Show' 'History' 'Horror'
 'Music' 'Musical' 'Mystery' 'News' 'Reality-TV' 'Romance' 'Sci-Fi'
 'Short' 'Sport' 'Talk-Show' 'Thriller' 'War' 'Western']
26


In [275]:
print(type(data['genre'][131]))

<class 'list'>


In [276]:
data.dtypes

genre           object
imdb_id         object
imdb_rating    float64
imdb_votes     float64
rated           object
released        object
runtime         object
title           object
year            object
ebay             int64
etsy             int64
bonanza          int64
etsy2          float64
dtype: object

In [303]:
def truncate_sql(dflisty):
    '''
    takes in a tv show dataframe then sends the data into AWS tv show database
    first doing the merchendise data
    '''
    cnx = mysql.connector.connect(
    #connect to mysql platform        
        host =host,
        user =user,
        passwd = pword
        )
    cur=cnx.cursor()
    cur.execute('USE tv_merch')
#      first pass the show data into sql
         cur.execute("""TRUNCATE TABLE show_data"""
#      then pass the sales data into sql
#         cur.execute("""INSERT INTO merch_data (
#                                     show_id,
#                                     etsy,
#                                     ebay,
#                                     bonanza,
#                                     show_name
#                                     ) 
#         VALUES ({},{},{},{},{}
#         )""".format(show['imdb_id'],
#                     show['etsy'],
#                     show['ebay'],
#                     show['bonanza'],
#                     show['title']))
    cnx.commit()
    cur.close()
    cnx.close()

In [341]:
data

,genre,imdb_id,imdb_rating,imdb_votes,rated,released,runtime,title,year,ebay,etsy,bonanza,etsy2
0,[Drama],tt1719758,3.9,123.0,NaN,11 May 2015,NaN,America's Court with Judge Ross,2010–,0,0,0,0.0
1,"[Documentary, Reality-TV]",tt1596786,7.0,6254.0,TV-PG,18 Jan 2010,60 min,American Pickers,2010–,5,2,0,2.0
2,[Documentary],tt3021452,7.3,18.0,NaN,01 Dec 2010,NaN,Ancient Aliens: The Mission,2010–,0,405,0,NaN
3,"[Crime, Drama, Mystery, Thriller]",tt1595859,7.5,29313.0,TV-14,24 Sep 2010,43 min,Blue Bloods,2010–,98,19,10,19.0
4,"[Comedy, Music, Talk-Show]",tt1637574,8.1,20564.0,TV-14,08 Nov 2010,42 min,Conan,2010–,252,30,0,30.0
5,"[Documentary, Crime]",tt1688779,7.8,1145.0,TV-14,11 Jul 2010,44 min,"Drugs, Inc.",2010–,0,107,0,NaN
6,[Reality-TV],tt1800864,7.3,3822.0,TV-14,03 Dec 2010,45 min,Gold Rush: Alaska,2010–,1,10,0,10.0
7,"[Game-Show, Reality-TV]",tt1711420,6.6,477.0,NaN,15 Aug 2010,60 min,The Great Food Truck Race,2010–,0,18,0,18.0
8,"[Action, Crime, Drama]",tt1600194,7.3,62054.0,TV-14,20 Sep 2010,42 min,Hawaii Five-0,2010–,316,16,0,16.0
9,[Documentary],tt1832668,9.0,4713.0,TV-PG,25 Apr 2010,60 min,How the Universe Works,2010–,0,1908,0,NaN


In [335]:
type(float(data['imdb_rating'][32]))

float

In [ ]:
def ship_2_sql2(dflisty):
    '''
    takes in a tv show dataframe then sends the data into AWS tv show database
    first doing the merchendise data
    '''
    cnx = mysql.connector.connect(
    #connect to mysql platform        
        host =host,
        user =user,
        passwd = pword
        )
    cur=cnx.cursor()
    cur.execute('USE tv_merch')
#     for i, show in dflisty.iterrows():
#         print(i,show['title'])
# #      first pass the show data into sql
# #         cur.execute("""TRUNCATE TABLE show_data"""
# #      then pass the sales data into sql
#         cur.execute("""INSERT INTO show_data (
#                                     show_id,
#                                     show_name,
#                                     rated,
#                                     released,
#                                     imdb_rating,
#                                     imdb_votes
#                                     ) 
#         VALUES ("{}","{}","{}","{}",{},{}
#         )""".format(show['imdb_id'],
#                     show["title"],
#                     show['rated'],
#                     show['year'],
#                     show['imdb_rating'],
#                     show['imdb_votes']
#                        ))
    cnx.commit()
    cur.close()
    cnx.close()

In [ ]:
data

In [339]:
cnx = mysql.connector.connect(
    #connect to mysql platform        
        host =host,
        user =user,
        passwd = pword
        )
cur=cnx.cursor()

In [ ]:
cur.close()

In [342]:
data.dtypes

genre           object
imdb_id         object
imdb_rating    float64
imdb_votes     float64
rated           object
released        object
runtime         object
title           object
year            object
ebay             int64
etsy             int64
bonanza          int64
etsy2          float64
dtype: object